# Code pour reformater les vidéos au même nombre de frames pour faciliter l'extraction et le traitement

# 1. Imports

In [19]:
import cv2
import os
import numpy as np
from os.path import isfile, join
from moviepy.editor import VideoFileClip
import csv
import mediapipe as mp 

mp_drawing = mp.solutions.drawing_utils  # Drawing helpers
mp_holistic = mp.solutions.holistic  # Mediapipe Solutions

# 2. Paramétrer les dossiers

Paramètres de bases

In [20]:
path = 'données/datas_mots/' #path vers le dossier des mots
data_path = 'output/coords_csv'
mots=np.array(['adresse','affaire','aller','ami_amie','autre','beaucoup','bonjour','chocolat','comprendre','demander','dieu','donc','dormir','enceinte','faire','famille','finir','gens','heure','ils_elles','interdire','jamais','jour','laisser','lentement','marcher_marche','merci','mourir_mort','nous_on','nuit','ou','payer','penser','personne','peu','pleuvoir_pluie','portugal','pour','prendre','question','quoi','raison_connaissance','rencontrer_rencontre','rester','rien','rue_route','sac','soeur_nonne','soleil','surprendre_surprise','temps','toujours','travail','trop','trouver']) #choix des mots
nb_videos=20 #nb de vidéos par mot

#'adresse','affaire','aller','ami_amie','autre','beaucoup','bonjour','chocolat','comprendre','demander','dieu','donc','dormir','enceinte','faire','famille','finir','gens','heure','ils_elles','interdire','jamais','jour','laisser','lentement','marcher_marche','merci','mourir_mort','nous_on','nuit','ou','payer','penser','personne','peu','pleuvoir_pluie','portugal','pour','prendre','question','quoi','raison_connaissance','rencontrer_rencontre','rester','rien','rue_route','sac','soeur_nonne','soleil','surprendre_surprise','temps','toujours','travail','trop','trouver','turquie','vache','vivre_vie','voir','vouloir'

In [21]:
len(mots)

55

Calculer le nombre d'images moyen par vidéo pour avoir la référence

In [22]:
count=0
fichier =list([])
for mot in mots:
    fichier1=[]
    i=0
    for f in os.listdir(path):
        if i<nb_videos:
            file_name, file_ext = os.path.splitext(f)
            testmot = file_name.split('-')
            if (testmot[0]==mot):
                fichier1 += [path + file_name + file_ext]
                i+=1
    fichier.append(fichier1)
rang_mot=0
for mot in mots:
    for f in fichier[rang_mot]:
        cap = cv2.VideoCapture(f)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        count=count+length
    rang_mot+=1
moy=count/(len(fichier)*nb_videos)
nb_frame=int(moy)#nombre d'images par vidéos
print(nb_frame)

17


In [23]:
print(fichier)

[['données/datas_mots/adresse-00.mp4', 'données/datas_mots/adresse-01.mp4', 'données/datas_mots/adresse-02.mp4', 'données/datas_mots/adresse-03.mp4', 'données/datas_mots/adresse-04.mp4', 'données/datas_mots/adresse-05.mp4', 'données/datas_mots/adresse-06.mp4', 'données/datas_mots/adresse-07.mp4', 'données/datas_mots/adresse-08.mp4', 'données/datas_mots/adresse-09.mp4', 'données/datas_mots/adresse-10.mp4', 'données/datas_mots/adresse-11.mp4', 'données/datas_mots/adresse-12.mp4', 'données/datas_mots/adresse-13.mp4', 'données/datas_mots/adresse-14.mp4', 'données/datas_mots/adresse-15.mp4', 'données/datas_mots/adresse-16.mp4', 'données/datas_mots/adresse-17.mp4', 'données/datas_mots/adresse-18.mp4', 'données/datas_mots/adresse-19.mp4'], ['données/datas_mots/affaire-00.mp4', 'données/datas_mots/affaire-01.mp4', 'données/datas_mots/affaire-02.mp4', 'données/datas_mots/affaire-03.mp4', 'données/datas_mots/affaire-04.mp4', 'données/datas_mots/affaire-05.mp4', 'données/datas_mots/affaire-06.mp4

Pour avoir les paramètres dans le csv

In [24]:
num_coords = 75 # nombre de points du corps + main gauche + main droite
# for landmark in results.right_hand_landmarks.landmark:
#     print(landmark, landmark.value)
landmarks = ['class']
for i in range (1,21): 
    for val in range(1, num_coords+1):
        landmarks += ['x' + str (val) + '_' + str(i), 'y' + str (val) + '_' + str(i),
                    'z' + str (val) + '_' + str(i), 'v' + str (val) + '_' + str(i)]
with open('output/coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(
        f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

Création des dossiers pour stocker les images

In [25]:
for mot in mots: 
    for vid in range(nb_videos):
        try: 
            os.makedirs(os.path.join(data_path, mot, str(vid)))
        except:
            pass

# 3. Code

Fonction intermédiaire pour plus tard calculer le framerate nécessaire à l'obtention du bon nombre de frames

In [26]:
def duree_et_longueurCV(video):

    duree = video.get(cv2.CAP_PROP_POS_MSEC)
    longueur = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return duree, longueur
    
def dureeMPY(file):
    clip = VideoFileClip(file)
    return clip.duration

Fonction pour stocker une image

In [27]:
def getFrame(vidcap,sec,rang_video,count):
            vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
            hasFrames,image = vidcap.read()
            if hasFrames:
                if count<10:
                    numero = '0' + str(count)
                else :
                    numero = str(count)
                cv2.imwrite(data_path+'/' + mot+'/'+str(rang_video)+'/image'+numero+".jpg", image)     # save frame as JPG file
            return hasFrames

Fonction pour trouver le bon framerate afin d'extraire le bon nombre d'images

In [28]:
def load_video_and_find_framerate(fichier,nb_frame):
    video = cv2.VideoCapture(f)
    duree = dureeMPY(fichier)
    fr=video.get(cv2.CAP_PROP_FPS)
    if fr!=25:
        video.set(cv2.CAP_PROP_FPS, 25)
        duree = (25*duree)/fr
        fr=25
    framerate = (duree/fr)*(fr/nb_frame)
    #print(fr,duree,framerate)
    return video, framerate

Code pour stocker automatiquement toutes les images des mots nous intéressants

In [29]:
rang_mot=0
for mot in mots:
    #print(mot)
    rang_video=0
    for f in fichier[rang_mot]:
        #print(f)
        sec = 0
        count=1
        vidcap, framerate = load_video_and_find_framerate(f,nb_frame)
        success = getFrame(vidcap,sec,rang_video,count)
        while success:
            count = count + 1
            sec = sec + framerate
            success = getFrame(vidcap,sec,rang_video,count)
        #print(len(os.listdir(data_path+mot+'/'+str(rang_video)+'/')))
        rang_video+=1
    rang_mot+=1

OSError: MoviePy error: failed to read the duration of file données/datas_mots/rester-16.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --en

# Analyser les images

### Préliminaires (mêmes fonctions que dans export_coord_v2)

In [ ]:
def mediapipe_detection(image, model): #Fonction pour que mediapipe puisse détecter (car il ne travaille pas dans le même système de couleur que OpenCV)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
                            
def extract_keypoints(results):
    pose = list(np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4))
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = list(np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
    rh = list(np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))
    #return np.concatenate([pose, face, lh, rh])
    return (pose + lh + rh)

def Image_normaliser(img):
    mp_holistic = mp.solutions.holistic
    with mp_holistic.Holistic(static_image_mode=True) as holistic:
        results = holistic.process(img)
        Right_hand_mark = np.array([[res.x, res.y, res.z]for res in results.right_hand_landmarks.landmark])if results.right_hand_landmarks else np.zeros((21,3))
        Left_hand_mark = np.array([[res.x, res.y, res.z]for res in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21,3))
        pose = np.array([[res.x,res.y,res.z]for res in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros((33,3))
    
        #on fait les calcule pour avoir les valeur des distance dans le monde pixel
    
        depx =  abs(pose[11][0] - pose[12][0]) #distance epaule 
        epaule1 = np.array(pose[12][0:3])
        epaule2 = np.array(pose[11][0:3])
        origine=(epaule1+epaule2)/2
        dornose=abs(pose[0][1]-origine[1])
        dz=depx  #comme on sait pas encore comment est estimé la valeur z on prend meme valeur que x
        dnormaliser=np.array([depx,dornose,dz])
        normaliser=np.array([40,20,40])
    
        shifted_hand_marks_coordr=Right_hand_mark-origine #right
        shifted_hand_marks_coordl=Left_hand_mark-origine  #left
        shifted_pose_marks_coord=pose-origine             #pose 
    
        hand_marks_coord_normalizedr = shifted_hand_marks_coordr*normaliser/dnormaliser  #right hand
        hand_marks_coord_normalizedl = shifted_hand_marks_coordl*normaliser/dnormaliser  #left hand
        pose_marks_coord_normalized = shifted_pose_marks_coord*normaliser/dnormaliser    #pose
   #     print(hand_marks_coord_normalizedr)
        #les valeurs retournées ne sont pas sous forme de vecteur utilise methode .flatten() pour vecteur.
        return list(np.concatenate(( (hand_marks_coord_normalizedr/np.linalg.norm(hand_marks_coord_normalizedr[...,:-1],axis=1).reshape(21,1)).flatten(), (hand_marks_coord_normalizedl/np.linalg.norm(hand_marks_coord_normalizedl[...,:-1],axis=1).reshape(21,1)).flatten(), (pose_marks_coord_normalized/np.linalg.norm(pose_marks_coord_normalized[...,:-1],axis=1).reshape(33,1)).flatten() ), axis=None))

Vérification

In [ ]:
print(nb_videos,nb_frame)

20 22


### Code

Fonction retournant la liste des fichiers d'un dossier

In [ ]:
def listfichier(path):
    L = os.listdir(path)
    L1 = L
    i=0
    for j in L:
        L1[i]  = path + '/' + L[i]
        i+=1
    #print(L1)
    return L1

In [ ]:
rang_mot = 0
rang_video = 0
rang_frame=0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    données = listfichier(data_path)
    for mot in données:
        dossiers=listfichier(mot + '/')
        for d in dossiers:
            fichiers=listfichier(d + '/')
            print(len(fichiers))
            row = []
            for f in fichiers:
                cap = cv2.VideoCapture(f)
                ret, frame = cap.read()
                if ret :
                    image, results = mediapipe_detection(frame, holistic)

                    draw_styled_landmarks(image, results)

                    row += Image_normaliser(frame)
                    
                    rang_frame+=1
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                cap.release()
                cv2.destroyAllWindows()
            row.insert(0, mots[rang_mot] )
            with open('output/coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            rang_frame=0
            rang_video+=1
        rang_video=0
        rang_mot+=1

22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22


In [ ]:
print(données)

['output/coords_csv/nous_on']


Supprimer les images intermédiaires

In [ ]:

for mot in données:
        dossiers=listfichier(mot + '/')
        for d in dossiers:
            fichiers=listfichier(d + '/')
            for f in fichiers:
                if f[-1]=='g':
                    os.remove(f) #supprimer l'image
            os.rmdir(d)
        os.rmdir(mot)
os.rmdir(data_path)
